# Statistics

A standard task in large-scale data processing, is to compute some statistics from a dataset. In the tutorial part, we will experiment on small samples, for the purpose of transparency in the processing. In the exercises, we will tackle a large dataset to make processing a bit more realistic.

In [ ]:
data = sc.parallelize([('M', 25), 
                       ('M', 20), 
                       ('M', 30), 
                       ('F', 25),
                       ('F', 20),
                       ('M', 30)])
values = data.map(lambda x:x[1])

## Basic statistics

There are a few basic operations, such as **count**ing the number of elements, taking the **sum** and the **mean** (average) of values. Since the mean is equal to the sum divided by the number of elements, sum/count will also give the mean.

In [ ]:
data.count()

In [ ]:
values.sum()

In [ ]:
values.mean()

In [ ]:
values.sum()/data.count()

## Standard descriptive statistics

There are also standard functions on RDDs to compute the standard deviation (average difference to the mean) and to distribute the values over buckets to prepare the data to plot in a histogram. For the standard deviation there are two functions, when working with sampled data (i.e. if we cannot observe all cases), the **sampleStddev** is more appropriate. 

In [ ]:
values.sampleStdev()

To plot a histogram, the data is distributed over equally sized buckets, and for every bucket the frequency represents the number of values that fall into that bucket. When we distribute the data over two equally sized buckets, the first list gives the bucket boundaries (20-25) and (25-30), and the second list the frequencies, e.g. 2 values fall into the (20-25) bucket (for all buckets except the last the upper bound is exclusive).

In [ ]:
values.histogram(2)

## Aggregations

For aggregated counts over keys and values, you can use the **countByKey** action on (key, value) pairs, and the **countByValue** action over elements.

In [ ]:
data.countByKey()

In [ ]:
values.countByValue()

If you want to do a different computation over keys, then the  **reduceByKey**, **aggregateByKey**, **combineByKey** and **groupByKey** are the transformation function that allow you to do that. When the sum, min or max over values are required, reduceByKey offers the most straightforward solution. But for more complex function the other transformations provide all the tools you need. 

In [ ]:
results = data.reduceByKey(lambda x, y: max(x, y))
print(results.collect())

## Sampling ##

Sampling takes a random sample of the data, much smaller than the original collection, to speed up processing. If the sample is representative for the entire collection, i.e. all occurrences and relations that are present in the collection are represented in equal proportions in the sample, the results of any process on the sample will be comparable to those on the entire collection. If the sample is close enough, this will still give a good indication of the results we can expect on the entire collection, and thus make data analysis more efficient.

In [1]:
filename = '../data/2008.csv.bz2'
if not os.path.exists(filename):
    import urllib.request
    urllib.request.urlretrieve ("http://stat-computing.org/dataexpo/2009/2008.csv.bz2", \
                                filename)

In [2]:
f = sqlContext.read.format("com.databricks.spark.csv").\
    options(header="true", inferSchema = "true").load(filename)

In [8]:
sample = f.sample(False, 0.01).cache()

In [5]:
%%time
f.groupBy(f.FlightNum).count().take(5)

CPU times: user 20 ms, sys: 55.2 ms, total: 75.2 ms
Wall time: 40.1 s


[Row(FlightNum=1580, count=2774),
 Row(FlightNum=463, count=2302),
 Row(FlightNum=2366, count=1142),
 Row(FlightNum=496, count=1946),
 Row(FlightNum=3918, count=618)]

In [9]:
%%time
sample.groupBy(f.FlightNum).count().take(5)

CPU times: user 11 ms, sys: 17.9 ms, total: 29 ms
Wall time: 50.5 s


[Row(FlightNum=1580, count=23),
 Row(FlightNum=1342, count=9),
 Row(FlightNum=148, count=27),
 Row(FlightNum=496, count=15),
 Row(FlightNum=3749, count=14)]